In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
# Base URL of the jobs page
base_url = "https://careers.servicenow.com/jobs/"

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [3]:

jobs = []

# Scraping job details from the ServiceNow careers page
for page in range(1, 11):  # Adjust range for more pages
    url = f"{base_url}?page={page}#results"
    response = requests.get(url, headers=headers)
    code = response.status_code
    soup = BeautifulSoup(response.content, "html.parser")

    for search in soup.find_all("div", class_="card card-job"):
        # Extract job name, location, and description link
        name = search.find("h2", class_="card-title")
        loc = search.find("li", class_="list-inline-item")
        desc = search.find("a", href=True)

        job_name = name.get_text(strip=True) if name else "NONE"
        job_loc = loc.get_text(strip=True) if loc else "NONE"
        job_desc = f"https://careers.servicenow.com{desc['href']}" if desc else "not mentioned"

        qualifications = []
        experience = "not mentioned"

        if job_desc != "not mentioned":
            try:
                # Fetch detailed job description page
                in_response = requests.get(job_desc, headers=headers)
                in_soup = BeautifulSoup(in_response.content, "html.parser")

                qualifications_text = ""
                for quest in in_soup.find_all("div", class_="js-fix-on-scroll-wrapper"):
                    # Find "Qualifications" section
                    quali_h3 = quest.find("h3", string="Qualifications")
                    if quali_h3:
                        next_tag = quali_h3.find_next(["ul", "p"])
                        while next_tag:
                            qualifications_text += next_tag.get_text(separator=" ", strip=True)
                            next_tag = next_tag.find_next(["ul", "p"])

                    # Handle "Preferred Qualifications" section
                    quali_p_strong = quest.find(
                        "p", string=lambda text: text and "Preferred Qualifications" in text
                    )
                    if quali_p_strong:
                        next_tag = quali_p_strong.find_next(["ul", "p"])
                        while next_tag:
                            qualifications_text += next_tag.get_text(separator=" ", strip=True)
                            next_tag = next_tag.find_next(["ul", "p"])

                # Normalize text (remove non-breaking spaces)
                qualifications_text = qualifications_text.replace("\xa0", " ")

                if qualifications_text:
                    qualifications.append(qualifications_text)

                    # Extract experience using regex
                    experience_pattern = re.compile(
                        r'\b(?:at least\s*\d+\+?\s*years?|minimum\s*\d+\+?\s*years?|\d+\+?\s*years?|one year|two years?|three years?|four years?|five years?|six years?|seven years?|eight years?|nine years?|ten years?)\b',
                        re.IGNORECASE,
                    )
                    experience_matches = experience_pattern.findall(qualifications_text)
                    experience = ', '.join(dict.fromkeys(experience_matches))

            except Exception as e:
                print(f"Error fetching details for {job_desc}: {e}")

        if experience.strip() == "":
            experience = "not mentioned"
        # Append job details to the list
        jobs.append(
            {
                "Job": job_name,
                "Location": job_loc,
                "Experience": experience,
                "Job Description": job_desc,
                "ServiceNow Page": page,
            }
        )
    print(f"ServiceNow Page: {page} :: status code {code} :: {url}")

# Display the scraped data
for job in jobs:
    print(job)


ServiceNow Page: 1 :: status code 200 :: https://careers.servicenow.com/jobs/?page=1#results
ServiceNow Page: 2 :: status code 200 :: https://careers.servicenow.com/jobs/?page=2#results
ServiceNow Page: 3 :: status code 200 :: https://careers.servicenow.com/jobs/?page=3#results
ServiceNow Page: 4 :: status code 200 :: https://careers.servicenow.com/jobs/?page=4#results
ServiceNow Page: 5 :: status code 200 :: https://careers.servicenow.com/jobs/?page=5#results
ServiceNow Page: 6 :: status code 200 :: https://careers.servicenow.com/jobs/?page=6#results
ServiceNow Page: 7 :: status code 200 :: https://careers.servicenow.com/jobs/?page=7#results
ServiceNow Page: 8 :: status code 200 :: https://careers.servicenow.com/jobs/?page=8#results
ServiceNow Page: 9 :: status code 200 :: https://careers.servicenow.com/jobs/?page=9#results
ServiceNow Page: 10 :: status code 200 :: https://careers.servicenow.com/jobs/?page=10#results
{'Job': 'Product Marketing Manager, Mid-Market', 'Location': 'Santa 

In [4]:
    import pandas as pd

In [5]:
job_table = pd.DataFrame(jobs)
job_table.index = job_table.index + 1
job_table.tail(25)

,Job,Location,Experience,Job Description,ServiceNow Page
176,"Senior Manager, Global Sales Development",Tokyo,"5+ years, 2+ years",https://careers.servicenow.com/jobs/7440000433...,9
177,"Engagement Lead, Global Talent Outreach & Enga...",Tokyo,not mentioned,https://careers.servicenow.com/jobs/7440000433...,9
178,Sr Enterprise Account Executive - Manufacturing,Tokyo,10+ years,https://careers.servicenow.com/jobs/7440000433...,9
179,Enterprise Account Executive - Manufacturing,Tokyo,7+ years,https://careers.servicenow.com/jobs/7440000433...,9
180,Sr Sales Development Representative,Tokyo,1+ year,https://careers.servicenow.com/jobs/7440000433...,9
181,Associate Account Executive,Tokyo,5+ years,https://careers.servicenow.com/jobs/7440000433...,10
182,Advisory Solution Consultant - Security & Risk,Tokyo,7+ years,https://careers.servicenow.com/jobs/7440000433...,10
183,Solution Sales Executive - Technology Workflow,Tokyo,5+ years,https://careers.servicenow.com/jobs/7440000433...,10
184,Enterprise Sales Associate,Tokyo,not mentioned,https://careers.servicenow.com/jobs/7440000433...,10
185,Sales Development Representative,Tokyo,not mentioned,https://careers.servicenow.com/jobs/7440000433...,10


In [6]:
job_table.shape

(200, 5)

In [7]:
#excelsheet = job_table.to_excel("SN_report.xlsx", index=False)

In [8]:
filtered_job_table = job_table[job_table['Experience'] == 'not mentioned']
filtered_job_table

,Job,Location,Experience,Job Description,ServiceNow Page
27,Industrial/Organizational Psychology Intern - ...,Dublin,not mentioned,https://careers.servicenow.com/jobs/7440000439...,2
40,"Strategic Partner Insights, Junior Specialist",Hyderabad,not mentioned,https://careers.servicenow.com/jobs/7440000438...,2
60,Mid-Market Account Executive,Staines,not mentioned,https://careers.servicenow.com/jobs/7440000437...,3
65,OTC Customer Support Analyst,Heredia,not mentioned,https://careers.servicenow.com/jobs/7440000437...,4
66,Senior Customer Success Manager - German speaker,Munich,not mentioned,https://careers.servicenow.com/jobs/7440000437...,4
68,ServiceNow Futures Director (Santa Clara or NY...,Santa Clara,not mentioned,https://careers.servicenow.com/jobs/7440000437...,4
70,Principal Data Platform Software Engineer,Santa Clara,not mentioned,https://careers.servicenow.com/jobs/7440000437...,4
71,Senior Principal Technical Program Manager,San Diego,not mentioned,https://careers.servicenow.com/jobs/7440000437...,4
80,Global Partner Manager,Austin,not mentioned,https://careers.servicenow.com/jobs/7440000436...,4
84,"Senior Manager, Enterprise Solution Architect",District Of Columbia,not mentioned,https://careers.servicenow.com/jobs/7440000436...,5


In [ ]:
#excelsheet2 = filtered_job_table.to_excel("notmentioned_SN_report.xlsx", index=False)